## <center> D214 - Capstone
<center> Kelseyann Wright 

## SET UP ENVIRONMENT & LOAD DATA

In [ ]:
#import basic libraries
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample


In [ ]:
#Load and preview the data
dat = pd.read_json("~/Desktop/WGU/D214/jeopardyquestions.json") #load data
dat.info()
dat.head()

## PREPROCESSING

In [ ]:

#Check for duplicates and nulls, remove if necessary
print("number of duplicates:", dat.duplicated().sum()) 
print("number of nulls:", dat.isna().sum().sum())
dat.dropna(axis=0, inplace = True) #drop NAs
print('AFTER DROPPING NANs: ')
dat.info()

In [ ]:
#remove rows with values not in normal play 
dat['value'] = dat['value'].str.replace(',', '') #remove commas
dat = dat.loc[dat['value'].isin(['$200', '$400', '$600', '$800', '$1000', '$1200', '$1600', '$2000'])] #keep only rows with these values
#create column with numeric values
dat['val_num'] = dat['value'].str.replace('$','').astype(float)

In [ ]:
plt.figure(figsize=(10,6))
fig = sns.countplot(data=dat, x='val_num', pallete = 'crest') #plot univariate barplot
plt.xticks(rotation = 90)
plt.show()

In [ ]:
#BALANCE DATA BY DOWN-SAMPLING LARGER GROUPS
df_bal = pd.DataFrame(columns = dat.columns)
for v in dat['value'].unique():
    df_vals = dat[dat.value == v] #create new dataframe with only thoes values
    df_downsampled = resample(df_vals, replace=False, n_samples=5000, random_state=55) #downsample
    df_bal = pd.concat([df_bal, df_downsampled], ignore_index = True)
    
dat = [] #clear dat out of memory
df_bal.info()


In [ ]:
plt.figure(figsize=(10,6))
fig = sns.countplot(data=df_bal, x='val_num') #plot univariate barplot
plt.xticks(rotation = 90)
plt.show()